<a href="https://colab.research.google.com/github/Mlakshay01/BERT-NER-FINE-TUNED/blob/main/CoNLL_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers datasets seqeval -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

Saving eng.testa to eng.testa
Saving eng.testb to eng.testb
Saving eng.train to eng.train
User uploaded file "eng.testa" with length 827443 bytes
User uploaded file "eng.testb" with length 748095 bytes
User uploaded file "eng.train" with length 3283420 bytes


In [ ]:
def parse_conll_file(filepath):
    tokens = []
    ner_tags = []
    all_data = []

    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip() == "":
                if tokens:
                    all_data.append({"tokens": tokens, "ner_tags": ner_tags})
                    tokens = []
                    ner_tags = []
            else:
                parts = line.strip().split()
                if len(parts) == 4:
                    tokens.append(parts[0])
                    ner_tags.append(parts[3])

    if tokens:
        all_data.append({"tokens": tokens, "ner_tags": ner_tags})

    return all_data


In [ ]:
train_data = parse_conll_file("/content/eng.train")
val_data = parse_conll_file("/content/eng.testa")
test_data = parse_conll_file("/content/eng.testb")


In [ ]:
from datasets import Dataset, DatasetDict

# 1. Extract label list and create mappings
all_labels = set(tag for sample in train_data for tag in sample["ner_tags"])
label_list = sorted(list(all_labels))
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

# 2. Encode string tags to numeric IDs
def encode_tags(example):
    example["ner_tags"] = [label2id[tag] for tag in example["ner_tags"]]
    return example

# 3. Create HuggingFace Datasets
train_dataset = Dataset.from_list(list(map(encode_tags, train_data)))
val_dataset = Dataset.from_list(list(map(encode_tags, val_data)))
test_dataset = Dataset.from_list(list(map(encode_tags, test_data)))

# 4. Combine into a DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Skip special tokens
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/14987 [00:00<?, ? examples/s]

Map:   0%|          | 0/3466 [00:00<?, ? examples/s]

Map:   0%|          | 0/3684 [00:00<?, ? examples/s]

In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00


In [ ]:
!pip install -q seqeval
import evaluate

metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)

    true_predictions = [
        [id2label[p] for (p, l) in zip(pred, lab) if l != -100]
        for pred, lab in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(pred, lab) if l != -100]
        for pred, lab in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }


In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./ner-bert",
    do_train=True,
    do_eval=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",  # logs for TensorBoard (optional)
    save_steps=500,        # how often to save
    eval_steps=500,        # how often to evaluate
    logging_steps=500,     # how often to log metrics
    report_to="none"
)


In [ ]:
!pip uninstall -y wandb


Found existing installation: wandb 0.20.1
Uninstalling wandb-0.20.1:
  Successfully uninstalled wandb-0.20.1


In [ ]:
from transformers import Trainer, DataCollatorForTokenClassification

# Collator handles padding dynamically
data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics  # this is the function we defined using seqeval
)

trainer.train()


/tmp/ipython-input-29-2831269781.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.061000
1000,0.074600
1500,0.039400
2000,0.030000
2500,0.018800
3000,0.017000
3500,0.010700
4000,0.009700
4500,0.006500


TrainOutput(global_step=4685, training_loss=0.028838870670395957, metrics={'train_runtime': 988.3769, 'train_samples_per_second': 75.816, 'train_steps_per_second': 4.74, 'total_flos': 1850028194927898.0, 'train_loss': 0.028838870670395957, 'epoch': 5.0})

In [ ]:
results = trainer.evaluate(tokenized_datasets["test"])

print("\n📊 Final Evaluation on Test Set:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")



📊 Final Evaluation on Test Set:
eval_loss: 0.1990
eval_precision: 0.8876
eval_recall: 0.9044
eval_f1: 0.8960
eval_accuracy: 0.9715
eval_runtime: 11.7228
eval_samples_per_second: 314.2600
eval_steps_per_second: 19.7050
epoch: 5.0000


In [ ]:
import torch

# Get device of model (GPU or CPU)
device = next(model.parameters()).device

# Your sentence
test_text = "Akshay studies in Delhi and works at Google."

# Tokenize and move to device
inputs = tokenizer(test_text, return_tensors="pt").to(device)

# Get model predictions
with torch.no_grad():
    outputs = model(**inputs).logits
predictions = torch.argmax(outputs, dim=2)

# Convert tokens back
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0].cpu())

# Show predicted labels (ignore special tokens)
print("\n🔍 Prediction on Custom Sentence:")
for token, pred in zip(tokens, predictions[0]):
    if token in ["[CLS]", "[SEP]", "[PAD]"]:
        continue
    print(f"{token:12} → {id2label[pred.item()]}")



🔍 Prediction on Custom Sentence:
A            → B-PER
##ks         → B-PER
##hay        → B-PER
studies      → O
in           → O
Delhi        → B-LOC
and          → O
works        → O
at           → O
Google       → B-ORG
.            → O
